In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import numpy as np
import pandas as pd
import os
import sys
import datetime
from pathlib import Path
import os
from absl import logging

sys.path.append('Enviroment')
from tf_trade_enviroment import MyTradeEnv

from tf_agents.environments import utils
from tf_agents.environments import tf_py_environment

from tf_agents.agents.dqn import dqn_agent
from tf_agents.networks import q_rnn_network
from tf_agents.utils import common
from tf_agents.trajectories import time_step as ts

from learningClass import learningHelper


logging.set_verbosity(logging.INFO)
tf.random.set_seed(12)
tf.print(tf.config.list_physical_devices('GPU') )
tf.print(tf.__version__)
tf.compat.v1.enable_v2_behavior()



[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]
2.3.1


In [2]:
from backtesting import Backtest, Strategy

Loading BokehJS ...

In [10]:
from sklearn.preprocessing import MinMaxScaler
from tf_agents.trajectories import time_step as ts

def convertDataToTimeStep(data, step_type):
    
    # scale price data
    data['price'] = MinMaxScaler().fit_transform(data['price']).astype(np.float32)

    #create time_step
    st = tf.constant(np.array(np.array([step_type], dtype=np.int32)))
    rw = tf.constant(np.array(np.array([0], dtype=np.float32)))
    ds = tf.constant(np.array(np.array([0], dtype=np.float32)))
    
    ts_obs = {}
    for key, value in data.items():
        ts_obs[key] = tf.expand_dims(tf.convert_to_tensor(value), axis=0)
    
    t = ts.TimeStep(st, rw, ds, ts_obs)

    return t


def runPolicy(t, _policy, _policy_state):
    
    if( t.step_type.numpy()[0] == 0):
        # new episode sarts -> reset policy state
        _policy_state = _policy.get_initial_state(1)

    a, _policy_state, _ = _policy.action(t, _policy_state)

    return a, _policy_state

def srtAction(action, isPosOpen):
    ret = 'None'
    if action == 0:
        ret = 'Skip'

    elif action == 1:
        if isPosOpen == True:
            ret = 'Buy -> Skip'
        else:
            ret = 'Buy'

    elif action == 2:
        if isPosOpen == True:
            ret = 'Sell -> Skip'+str(isPosOpen)
        else:
            ret = 'Sell'

    elif action == 3:
        if isPosOpen == False:
            ret = 'Close -> Skip'
        else:
            ret = 'Close'

    return ret

In [69]:
class TestPolicyStrategy(Strategy):
    def init(self):
        self.counter = 0
        self.price_data = np.zeros((20,5))
        self.pos = [0, 0]
        self.collet_number = 10
        self.max_number = 20
        self.isPosOpen = False

        policy_dir = './rnn_chkp/policy/'
        self.policy = tf.compat.v2.saved_model.load(policy_dir)
        self.policy_state = self.policy.get_initial_state(1)

        pass

    def next(self):

        #print(self.counter)
        #collect data before trading
        if (self.counter < self.max_number):
            x = np.array([self.data.High[-1], self.data.Low[-1], self.data.Open[-1], self.data.Close[-1], self.data.Volume[-1]])
            self.price_data[self.counter] = x.copy()

        if(self.counter >= self.collet_number):
            observation = {
                'price': self.price_data,
                'pos'  : self.pos
                }
            
            if(self.counter == self.collet_number):
                t = convertDataToTimeStep(observation, 0)  # initial time step
            elif(self.counter == self.max_number):
                t = convertDataToTimeStep(observation, 2) # last time step
            else:
                t = convertDataToTimeStep(observation, 1) 

            #print(t)
            a, self.policy_state = runPolicy(t, self.policy, self.policy_state)

            #print('action: {} [{}] -> step_type: {}'.format(a.numpy(), srtAction(a.numpy()[0], self.isPosOpen), t.step_type.numpy()))

            if a == 1:
                self.pos[0] = 1
            elif a == 2:
                self.pos[1] = 1
            elif a == 3:
                self.pos[0] = 0
                self.pos[1] = 0

            if self.isPosOpen == True:
                if a == 3:
                    self.isPosOpen = False
            else:
                if a == 1 or a == 2:
                    self.isPosOpen = True
                    if a==1:
                        self.buy()
                        #print('buy')
                    if a==2:
                        self.sell()
                        #print('sell')
                
        self.counter +=1

        if(self.counter > self.max_number):
           self.counter = self.collet_number
           self.price_data[:self.collet_number] = self.price_data[-self.collet_number:]
           self.price_data[self.collet_number:] = 0
           self.pos = [0, 0]
           self.isPosOpen = False

In [102]:
import pandas_datareader as pdr
begin_data = datetime.datetime(2020, 1, 1)
end_data = datetime.datetime(2020, 12, 1)

pd_data = pdr.get_data_yahoo('AMD', start=begin_data, end=end_data)

pd_data = pd_data[['Open','High','Low','Close','Volume']]
pd_data.head()

,Open,High,Low,Close,Volume
Date,,,,,
2020-01-02,46.860001,49.250000,46.630001,49.099998,80331100
2020-01-03,48.029999,49.389999,47.540001,48.599998,73127400
2020-01-06,48.020000,48.860001,47.860001,48.389999,47934900
2020-01-07,49.349998,49.389999,48.040001,48.250000,58061400
2020-01-08,47.849998,48.299999,47.139999,47.830002,53767000


In [103]:
bt = Backtest(pd_data, TestPolicyStrategy, commission=.002,
              exclusive_orders=True, cash=10000, trade_on_close=True)
stats = bt.run()
bt.plot()

In [104]:
stats

Start                     2020-01-02 00:00:00
End                       2020-12-01 00:00:00
Duration                    334 days 00:00:00
Exposure Time [%]                     78.0172
Equity Final [$]                       6015.6
Equity Peak [$]                       13479.6
Return [%]                            -39.844
Buy & Hold Return [%]                 88.6558
Return (Ann.) [%]                    -42.4227
Volatility (Ann.) [%]                 32.2275
Sharpe Ratio                                0
Sortino Ratio                               0
Calmar Ratio                                0
Max. Drawdown [%]                    -57.5068
Avg. Drawdown [%]                    -11.8724
Max. Drawdown Duration      260 days 00:00:00
Avg. Drawdown Duration       39 days 00:00:00
# Trades                                   29
Win Rate [%]                          44.8276
Best Trade [%]                        16.9633
Worst Trade [%]                      -25.7507
Avg. Trade [%]                    